In [5]:
from gensim.models import Word2Vec
model = Word2Vec.load('model.pkl')

In [2]:
import utils
word = 'горшок'
weights = utils.load_weights('../corpora/ad-nouns/cdict/', word)
contexts = utils.load_contexts('../corpora/ad-nouns-contexts-100k', word)
print(contexts[0])
print(len(contexts))

['час', 'время', 'инцидент', 'хватить', 'некоторый', 'легенда', 'говориться', 'золото', 'находиться', 'конец', 'радуга', 'глубина', '22', 'фут', 'почва', 'становиться']
34188


In [12]:
from collections import Counter
import numpy as np
min_count = 20
min_weight = 1.5
all_words = [
    w for w, cnt in Counter(w for ctx in contexts for w in ctx).items()
    if cnt >= min_count and weights.get(w, 0) > 1.5]
print(len(all_words))
w2v_vecs = np.array([model[w] for w in all_words if w in model])
print(len(w2v_vecs))

679
678


In [13]:
import kmeans
n_senses = 6
km = kmeans.KMeans(w2v_vecs, k=n_senses, metric='cosine')

kmeans: X (60, 300)  centres (6, 300)  delta=0.001  maxiter=10  metric=cosine
kmeans: 5 iterations  cluster sizes: [ 3 18 16  7  6 10]
kmeans: X (678, 300)  centres (6, 300)  delta=0.001  maxiter=10  metric=cosine
kmeans: 7 iterations  cluster sizes: [ 36 224  85 118  53 162]


In [17]:
all_words = np.array(all_words)
for sense in range(n_senses):
    indices = km.Xtocentre == sense
    distances = km.distances[indices]
    sense_words = all_words[indices]
    min_indices = np.argsort(distances)[:10]
    min_words = list(sense_words[min_indices])
    min_words.sort(key=lambda w: weights.get(w, 0), reverse=True)
    print(sense, ' '.join(min_words[:5]))

0 какашка испражнение карапуз годовалый отросток
1 глиняный плошка кадка кадушка керамический
2 сметана баранина суп вскармливание сало
3 герань бегония азалия кактус орхидея
4 вдребезги треснуть опрокидывать разбитый разбивать
5 сажать закапывать зарывать окунать отмывать
